In [ ]:
import pandas as pd
from drn_interactions.config import Config
from drn_interactions.interactions.popcup import PopulationCoupling
from drn_interactions.interactions.preprocessors import InteractionsPreprocessor
from drn_interactions.interactions.loaders import SpontaneousActivityLoader
from drn_interactions.interactions.pairwise import PairwiseCorr, PairwisePartialCorr
from drn_interactions.interactions.runners import InteractionsRunner
from drn_interactions.io import load_derived_generic
from drn_interactions.config import Config

dd = Config.derived_data_dir / "corrs"
dd.mkdir(exist_ok=True)

In [ ]:
neuron_types = load_derived_generic("neuron_types.csv")
sessions = neuron_types["session_name"].dropna().unique()

res_corr = []
res_pcorr = []
res_pcup = []
bin_widths = [0.05, 0.1, 0.5, 1, 2, 5]

loader_fac = lambda bin_width: SpontaneousActivityLoader(
    session_name=sessions[0], bin_width=bin_width
)
preprocessor_fac = lambda: InteractionsPreprocessor(
    z=True, minmax=False, gaussian_sigma=None
)
corr_fac = lambda: PairwiseCorr()
pcorr_fac = lambda: PairwisePartialCorr()
pcup_fac = lambda: PopulationCoupling()

runner_fac = lambda bin_width: InteractionsRunner(
    loader=loader_fac(bin_width),
    preprocessor=preprocessor_fac(),
    pcup=pcup_fac(),
    corr=corr_fac(),
    pcorr=pcorr_fac(),
)
for bin_width in bin_widths:
    runner = runner_fac(bin_width)
    res_pcorr.append(runner.run_pcorr_multi(sessions).assign(bin_width=bin_width))
    res_corr.append(runner.run_corr_multi(sessions).assign(bin_width=bin_width))
    res_pcup.append(runner.run_pcup_multi(sessions).assign(bin_width=bin_width))

res_corr = pd.concat(res_corr).reset_index(drop=True)
res_pcorr = pd.concat(res_pcorr).reset_index(drop=True)
res_pcup = pd.concat(res_pcup).reset_index(drop=True)

res_pcorr.to_csv(dd / "spont - pcorr.csv", index=False)
res_corr.to_csv(dd / "spont - corr.csv", index=False)
res_pcup.to_csv(dd / "spont - pcup.csv", index=False)


In [ ]:
runner = runner_fac(1)
pcorr_reps = runner.pcorr_bootstrap_multi(sessions=sessions, n_boot=100)
corr_reps = runner.corr_bootstrap_multi(sessions=sessions, n_boot=100)
pcup_reps = runner.pcup_bootstrap_multi(sessions=sessions, n_boot=100)

pcorr_reps.to_parquet(
    dd / "spont - pcorr_reps.parquet.gzip", index=False, compression="gzip"
)
corr_reps.to_parquet(
    dd / "spont - corr_reps.parquet.gzip", index=False, compression="gzip"
)
pcup_reps.to_parquet(
    dd / "spont - pcup_reps.parquet.gzip", index=False, compression="gzip"
)
